# Controller #

Import all neccessary packages

In [1]:
import os
import pandas as pd
import numpy as np

from drivers.sandbox import Sandbox
from drivers.generator import Generator
from drivers.optimiser import Optimiser

from inference_toolbox.domain import Domain
from inference_toolbox.parameter import Parameter
from inference_toolbox.model import Model
from inference_toolbox.likelihood import Likelihood

current_directory = os.getcwd()
if current_directory != '/project/':
    os.chdir('/project/')

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Define how the data is to be generated and what default inputs should be used

In [2]:
# data_params = {
#     'data_type': 'dummy',
#     'data_path': 'data',
#     'sigma': 1,
#     'model_select': 'log_gpm_alt_norm',
#     'noise_dist': 'gaussian',
#     'model': {
#         'model_params':{
#             'H': 10
#         },
#         'inference_params':{
#             'I_y': 0.1,
#             'I_z': 0.1,
#             'Q': 3e13,
#             'sigma': 1
#         },
#     },
#     'domain': {
#         'domain_select': 'cone_from_source_z_limited', 
#         'resolution': 20,
#         'domain_params':{
#             'r': 100,
#             'theta': np.pi/8,
#             'source': [0,0,10]}
#     },
#     'output_header': 'Concentration'
# }

data_params = {
    'data_type': 'gridded',
    'output_header': 'Concentration',
    'log':True,
    'grid_size': [100,100,25],
    'target': False,
    'data_path':'data'
}


default_params = {
    'infered_params':pd.Series({
        'model_params':pd.Series({
            'I_y': Parameter('I_y', prior_select = 'gamma', default_value=0.1).add_prior_param('mu', 0.1).add_prior_param('sigma',0.1),
            'I_z': Parameter('I_z', prior_select = 'gamma', default_value=0.1).add_prior_param('mu', 0.1).add_prior_param('sigma',0.1),
            'Q': Parameter('Q', prior_select = 'gamma', default_value=3e13).add_prior_param('mu', 3e13).add_prior_param('sigma',1e13),
        }),
        'likelihood_params':pd.Series({
            'sigma': Parameter('sigma', prior_select = 'gamma', default_value=1).add_prior_param('mu', 1).add_prior_param('sigma',1)
        })
    }),
    'model':Model('log_gpm_alt_norm').add_model_param('H',10),
    'likelihood': Likelihood('gaussian'),
    'sampler': {
        'n_samples': 10000,
        'n_chains': 3,
        'thinning_rate': 1
    }
}

results_name = 'gridded_data_100_100_25_real'

## Sandbox ##

Create an instance with the inputted default parameters and visualise it in different ways

In [3]:
sandbox = Sandbox(results_name=results_name, 
                  default_params=default_params, 
                  data_params=data_params)

visualiser = sandbox.run()
visualiser.get_summary()
visualiser.get_traceplot()
visualiser.get_autocorrelations()

domain = Domain('cone_from_source_z_limited', resolution=80)
domain.add_domain_param('r', 1000)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,10])

visualiser.visualise_results(domain = domain, name = 'small_scale_3D_plots', title='Log Concentration of Droplets', log_results=False)
visualiser.animate(name = 'small_scale_3D_plots')

Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Autocorrelations plot 1 already exists
Autocorrelations plot 2 already exists
Autocorrelations plot 3 already exists
Plots already exist!
Animation already exist!


## Generator ##

Generate different instances by varying the hyperparameters with respect to their defaults

In [4]:
generator = Generator(results_name=results_name, 
                  default_params=default_params, 
                  data_params=data_params)

Vary one hyperparameter

In [5]:
# analysis_iterations = {
#     'parameters':
#     [
#         'sigma_mu', 
#         'sigma_sigma'
#     ],
#     'values':
#     [
#         np.array([0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2]),
#         np.array([0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2]),
#     ]
# }

# for i in range(len(analysis_iterations['parameters'])):
#     parameter = analysis_iterations['parameters'][i]
#     print('Working on varying ' + parameter + '...')
#     values = analysis_iterations['values'][i]
#     inputs = generator.vary_one_parameter(parameter, values, xscale='linear', plot=True)

Vary two hyperparameters

In [6]:
analysis_iterations = {
    'parameters_1':
    [
        'sigma_mu',
         
    ],
    'parameters_2':
    [
        'sigma_sigma',
    ],
    'values_1':
    [
        np.array([0.4, 0.8, 1.2, 1.6, 2, 2.4, 2.8, 3.2, 3.6, 4])
    ],
    'values_2':
    [
        np.array([0.4, 0.8, 1.2, 1.6, 2, 2.4, 2.8, 3.2, 3.6, 4])
    ]
}

for i in range(len(analysis_iterations['parameters_1'])):
    parameter_1 = analysis_iterations['parameters_1'][i]
    parameter_2 = analysis_iterations['parameters_2'][i]

    print('Working on varying ' + parameter_1 + '...')
    values_1 = analysis_iterations['values_1'][i]
    values_2 = analysis_iterations['values_2'][i]
    inputs = generator.vary_two_parameters(parameter_1, parameter_2, values_1, values_2, scale_1='log', scale_2='log', plot=True)

Working on varying I_y_mu...
Working on varying I_z_mu...
Working on varying Q_mu...


## Optimiser ##

Optimise the default construction using Bayesian Optimisation

In [7]:
optimising_parameters = {
                    'I_y_mu': [1e-2, 10],
                    'I_y_sigma': [1e-2, 10],
                    'I_z_mu': [1e-2, 10],
                    'I_z_sigma': [1e-2, 10],
                    'Q_mu': [1e9, 1e18],
                    'Q_sigma': [1e9, 1e18],
                    'sigma_mu':[0.2, 2],
                    'sigma_sigma':[0.2, 2]
                }

optimiser = Optimiser(results_name=results_name, 
                  default_params=default_params, 
                  data_params=data_params)

study = optimiser.run(n_trials=100, optimiser_name='optimiser_1', optimising_parameters=optimising_parameters, index_name='aic')
optimiser.get_plots(study)

optimiser.run_best_params(study)

[I 2023-07-27 16:24:14,774] A new study created in memory with name: no-name-fc1fd927-aac4-43be-a52e-1ef577fb35ff
sample: 100%|██████████| 12500/12500 [00:04<00:00, 2999.43it/s, 15 steps of size 3.92e-01. acc. prob=0.92]
[I 2023-07-27 16:24:35,029] Trial 0 finished with value: 83.53607177734375 and parameters: {'I_y_mu': 4.61055628394951, 'I_y_sigma': 6.30158802521391, 'I_z_mu': 4.197865548850372, 'I_z_sigma': 0.3707295487249489, 'Q_mu': 9.037180455140252e+17, 'Q_sigma': 8.298234319609466e+17, 'sigma_mu': 1.768326271735175, 'sigma_sigma': 0.5980090196820267}. Best is trial 0 with value: 83.53607177734375.
sample: 100%|██████████| 12500/12500 [00:03<00:00, 3699.68it/s, 7 steps of size 7.51e-01. acc. prob=0.90]
[I 2023-07-27 16:24:51,869] Trial 1 finished with value: 79.43879699707031 and parameters: {'I_y_mu': 7.997301362470987, 'I_y_sigma': 0.13697693548032544, 'I_z_mu': 5.939157007560699, 'I_z_sigma': 2.2028061865171784, 'Q_mu': 3.756149662546317e+17, 'Q_sigma': 5.0765986021650355e+17

: 

: 